# GENERADOR DE DATASET- GASTOS PERSONALES

Como es de conocimeinto publico los datos de ingresos y gastos de una persona son información sensible y a falta de un dataset completo de gastos e ingresos de una persona se desarrolla este proyecto para las practicas de ciencia de datos.

## historial de remuneracion minima vital 

fuente (BCR): https://estadisticas.bcrp.gob.pe/estadisticas/series/mensuales/resultados/PN02124PM/html/1970-1/2025-9/

In [22]:
# Instalación de librerías necesarias
%pip install pandas numpy matplotlib seaborn scikit-learn statsmodels prophet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 3.4 MB/s  0:00:02m 3.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 2.6 MB/s  0:00:04 eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.9 MB/s  0:00:00m 3.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [prophet]━  9/10 [prophet]y]rn]ources]
Note: you may need to restart the kernel to use updated packages.


In [50]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [40]:
# --- 1. Cargar el archivo y configurar ---

# Nombre de los archivos
archivo_entrada = 'historial_salario_minimo.csv'
archivo_salida = 'proyeccion_salario_minimo.csv'

In [42]:
# Ruta absoluta del directorio donde se encuentra este script/notebook
directorio_actual = os.getcwd()

In [44]:
# Ruta completa del archivo de entrada
ruta_entrada = os.path.join(directorio_actual, archivo_entrada)

In [49]:
try:
    df = pd.read_csv(ruta_entrada)
except FileNotFoundError:
    print(f"Error: El archivo '{archivo_entrada}' no se encontró en el directorio: {directorio_actual}")
    exit()

# --- 2. Preprocesamiento y Creación de la Serie Temporal ---

# Función para convertir 'Ene91' a '1991-01-01'
def parse_date(date_str):
    mes = date_str[:3]
    anio = date_str[3:]
    meses_map = {
        'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Ago': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
    }
    # Completar el año (ej. '91' a '1991', '25' a '2025')
    if len(anio) == 2:
        if int(anio) > 50:
            anio = '19' + anio
        else:
            anio = '20' + anio
    return datetime(int(anio), meses_map[mes], 1)

# Aplicar la función de conversión de fecha
df['Fecha_dt'] = df['Fecha'].apply(parse_date)

# Establecer la fecha de inicio para el cálculo de los días (Ene91)
fecha_inicio = df['Fecha_dt'].min()

# Calcular el número de meses desde el inicio (característica temporal)
# Esto es X para el modelo de regresión
df['Mes_Index'] = (df['Fecha_dt'].dt.to_period('M') - fecha_inicio.to_period('M')).apply(lambda x: x.n)
df['Salario_minimo'] = pd.to_numeric(df['Salario_minimo'], errors='coerce')


# --- 3. Entrenamiento del Modelo (Regresión Polinomial de Grado 2) ---

X_train = df[['Mes_Index']].values
y_train = df['Salario_minimo'].values

# Crear características polinomiales (grado 2)
# Grado 2 es Mes_Index y (Mes_Index)^2
poly_features = PolynomialFeatures(degree=2)
X_poly_train = poly_features.fit_transform(X_train)

# Entrenar el modelo de Regresión Lineal con las características polinomiales
model = LinearRegression()
model.fit(X_poly_train, y_train)

# --- 4. Preparación de la Proyección (hasta Dic-2035) ---

# Calcular el índice del último mes en los datos originales (Dic2025)
ultimo_indice = df['Mes_Index'].max()

# Definir la fecha de fin de la proyección (Dic-2035)
fecha_fin_proyeccion = datetime(2035, 12, 1)

# Calcular el índice del último mes de la proyección
# LÍNEA CORREGIDA: Se usa pd.Timestamp() para convertir el objeto datetime estándar
indice_fin_proyeccion = (pd.Timestamp(fecha_fin_proyeccion).to_period('M') - fecha_inicio.to_period('M')).n

# Crear una serie de índices de meses para la proyección (desde el inicio hasta Dic-2035)
indices_proyeccion = np.arange(0, indice_fin_proyeccion + 1).reshape(-1, 1)

# Crear las características polinomiales para la proyección
X_proyeccion = poly_features.transform(indices_proyeccion)

# Realizar la proyección
proyecciones = model.predict(X_proyeccion)

# --- 5. Creación del DataFrame de Proyección y Exportación ---

# Crear el DataFrame de resultados
df_proyeccion = pd.DataFrame()
df_proyeccion['Mes_Index'] = indices_proyeccion.flatten()

# Crear la columna de Fecha en formato datetime
df_proyeccion['Fecha_dt'] = df_proyeccion['Mes_Index'].apply(
    lambda i: fecha_inicio + pd.DateOffset(months=i)
)

# Formatear la columna 'Fecha' como 'Ene91', 'Feb91', etc.
def format_date_output(dt):
    mes_abr = dt.strftime('%b')[:3]
    anio_dos_digitos = dt.strftime('%y')
    return f"{mes_abr.capitalize()}{anio_dos_digitos}"

df_proyeccion['Fecha'] = df_proyeccion['Fecha_dt'].apply(format_date_output)

# Asignar los salarios proyectados
df_proyeccion['Salario_minimo_proyectado'] = np.maximum(proyecciones, df['Salario_minimo'].min()).round(2)

# Determinar si el valor es un historial o una proyección
df_proyeccion['Tipo_de_valor'] = np.where(
    df_proyeccion['Mes_Index'] <= ultimo_indice, 
    'Historial', 
    'Proyección'
)

# Filtrar para obtener solo las columnas deseadas para la exportación
df_final = df_proyeccion[['Fecha', 'Salario_minimo_proyectado', 'Tipo_de_valor']].rename(columns={'Salario_minimo_proyectado': 'Salario_minimo'})

# Guardar el nuevo archivo CSV
ruta_salida = os.path.join(directorio_actual, archivo_salida)
df_final.to_csv(ruta_salida, index=False)

print(f"✅ ¡Proceso completado!")
print(f"Se ha creado el archivo de proyección: {archivo_salida}")
print(f"Ubicación: {ruta_salida}")
print("\n--- Vista previa de los datos proyectados (últimas filas) ---")
print(df_final[df_final['Tipo_de_valor'] == 'Proyección'].tail())

✅ ¡Proceso completado!
Se ha creado el archivo de proyección: proyeccion_salario_minimo.csv
Ubicación: /home/gandalf/Projects/notebooks/gastos_personales_dataset_generator/proyeccion_salario_minimo.csv

--- Vista previa de los datos proyectados (últimas filas) ---
     Fecha  Salario_minimo Tipo_de_valor
535  Aug35         1382.71    Proyección
536  Sep35         1385.17    Proyección
537  Oct35         1387.62    Proyección
538  Nov35         1390.08    Proyección
539  Dec35         1392.54    Proyección


In [ ]:
# --- 6. Visualización de los Resultados ---

print("\n--- Generando Gráfica de Proyección ---")

# Convertir la columna de fecha a formato numérico (para el eje X)
df_final['Mes_Num'] = df_final['Fecha_dt'].apply(lambda x: x.toordinal())

# Definir los datos históricos y de proyección
df_historial = df_final[df_final['Tipo_de_valor'] == 'Historial']
df_proyeccion = df_final[df_final['Tipo_de_valor'] == 'Proyección']
df_punto_corte = df_final.iloc[df_historial.index[-1]]

# Configuración de la gráfica
plt.figure(figsize=(16, 7))
sns.set_style("whitegrid")

# 1. Graficar la data histórica (línea sólida)
plt.plot(
    df_historial['Fecha_dt'], 
    df_historial['Salario_minimo'], 
    label='Historial (Ene/91 - Dic/25)', 
    color='darkblue', 
    linewidth=2.5
)

# 2. Graficar la data proyectada (línea punteada)
plt.plot(
    df_proyeccion['Fecha_dt'], 
    df_proyeccion['Salario_minimo'], 
    label='Proyección Polinomial (Ene/26 - Dic/35)', 
    color='red', 
    linestyle='--', 
    linewidth=2
)

# 3. Marcar el punto de corte
plt.scatter(
    df_punto_corte['Fecha_dt'], 
    df_punto_corte['Salario_minimo'], 
    color='gold', 
    edgecolor='black', 
    s=150, 
    zorder=5, 
    label='Último valor histórico (Dic/25)'
)

# 4. Añadir Títulos y Etiquetas
plt.title('Proyección del Salario Mínimo usando Regresión Polinomial (1991 - 2035)', fontsize=18, fontweight='bold')
plt.xlabel('Año', fontsize=14)
plt.ylabel('Salario Mínimo (Valor Monetario)', fontsize=14)
plt.legend(loc='upper left', fontsize=12)
plt.xticks(rotation=45, ha='right')

# 5. Formato del Eje X (Mostrar solo una selección de años para claridad)
# Seleccionar fechas representativas cada 4 años
fechas_ticks = pd.to_datetime([f'{y}-01-01' for y in range(1991, 2036, 4)])
plt.gca().set_xticks(fechas_ticks)
plt.gca().set_xticklabels([f.strftime('%Y') for f in fechas_ticks])


# 6. Guardar y Mostrar la gráfica
plt.tight_layout()
ruta_grafica = os.path.join(directorio_actual, 'proyeccion_salario_minimo.png')
plt.savefig(ruta_grafica)
plt.show()

print(f"Se ha guardado la gráfica en: {ruta_grafica}")